In [24]:
from __future__ import division

import os, platform, sys, operator as op
import numpy as np
import theano
import theano.tensor as T
import theano_models as tm
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
import warnings
from experiment_util import track, hyper_init_random, hyper_init_dict
from schlichtanders.mycontextmanagers import ignored
import csv
from ast import literal_eval
import random

from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

inf = float("inf")
warnings.filterwarnings("ignore", category=DeprecationWarning)
tm.inputting_references.update(['to_be_randomized'])

__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [29]:
def toy_likelihood():
    x = tm.as_tensor_variable([0.5])  #T.vector()
    y = x + 0.3 * T.sin(2*np.pi*x)
    func = tm.Model(inputs=[x], outputs=y, name="sin")
    return tm.Merge(pm.GaussianNoise(y, init_var=0.001), func, ignore_references={'parameters', 'parameters_positive'})

In [25]:
# defaults:
foldername = "test"
filename = "toy_example_test"
#overwrite as far as given:
if len(sys.argv) > 2:
    foldername, filename = sys.argv[1:3]
elif len(sys.argv) > 1:
    foldername = sys.argv[1]

with ignored(OSError):
    os.mkdir(os.path.join(__path__, foldername))
filepath = os.path.join(__path__, foldername, '%s.db' % filename)
errorfilepath = os.path.join(__path__, foldername, '%s_errors.txt' % filename)
csvpath = os.path.join(__path__, 'good_parameters.csv')


model_names = { # sorted by optimization type
    "ml": ['baselinedet'],
    "annealing": ['baseline', 'mixture', 'planarflow', 'planarflowdet', 'radialflow', 'radialflowdet'],
}
model_prefixes = reduce(op.add, model_names.values())
model_prefixes = [p+"_" for p in model_prefixes]

# Hyperparameters
# ===============

Base = declarative_base()

class Hyper(Base):
    __tablename__ = "hyper"
    id = Column(Integer, primary_key=True)

    # hyper parameters:
    x_true = Column(Float)
    max_epochs_without_improvement = Column(Integer)
    logP_average_n = Column(Integer)
    errorrate_average_n = Column(Integer)
    minus_log_s1 = Column(Integer)
    minus_log_s2 = Column(Integer)
    batch_size = Column(Integer)

    n_normflows = Column(Integer)

    opt_identifier = Column(String)
    opt_momentum = Column(Float)
    opt_offset = Column(Float)
    opt_decay = Column(Float)
    opt_step_rate = Column(Float)

    for _prefix in model_prefixes:
        exec("""
{0}best_val_loss = Column(Float)
{0}best_parameters = Column(PickleType, nullable=True)
{0}train_loss = Column(PickleType)
{0}val_loss = Column(PickleType)
{0}epochs = Column(Integer)
{0}init_params = Column(PickleType, nullable=True)
{0}val_error_rate = Column(Float)""".format(_prefix))
    def __init__(self, x_true):
        """
        Parameters
        ----------
        datasetname : str
        """
        self.x_true = x_true
        self.max_epochs_without_improvement = 30
        self.logP_average_n = 3  # TODO random.choice([1,10])
        self.errorrate_average_n = 10
        self.init_results()

    def init_results(self):

        # extra for being able to reset results for loaded hyperparameters
        for prefix in model_prefixes:
            setattr(self, prefix + "best_parameters", None)
            setattr(self, prefix + "best_val_loss", inf)
            setattr(self, prefix + "train_loss", [])
            setattr(self, prefix + "val_loss", [])
            setattr(self, prefix + "best_epoch", 0)
            setattr(self, prefix + "init_params", None)
            setattr(self, prefix + "val_error_rate", inf)

engine = create_engine('sqlite:///' + filepath)  # os.path.join(__path__, foldername, '%s.db' % filename)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
sql_session = Session()


In [26]:
good_parameters = []
with open(csvpath, "r") as f:
    reader = csv.DictReader(f, quoting=csv.QUOTE_NONE)
    for row in reader:
        # this should not be updated in hyper
        # (double quote as quoting=csv.QUOTE_NONNUMERIC was used to create this csv)
        del row['"datasetname"']
        # TODO if the first runs through, delete also n_normflows, as we want to have all of them
        good_parameters.append(
            {literal_eval(k): literal_eval(v) for k, v in row.iteritems()}
        )  # evaluate everything, this version also distinguishes ints/floats
random.shuffle(good_parameters)  # permutes inplace

In [33]:
for n_normflows in [1,2,3,4,8,20]:
    x_true = 0.63
    params = good_parameters[0]
    hyper = Hyper(x_true)
    hyper_init_random(hyper)
    hyper_init_dict(hyper, params)
    hyper.n_normflows = n_normflows

    targets = toy_likelihood()
    total_size = tm.total_size(targets['inputs'])
    params_base = pm.DiagGauss(output_size=total_size)
    normflows = [dm.PlanarTransform() for _ in range(hyper.n_normflows)]
    # LocScaleTransform for better working with PlanarTransforms
    params = params_base
    for transform in normflows:
        params = tm.normalizing_flow(transform, params)  # returns transform, however with adapted logP  # TODO merge does not seem to work correctly

    prior = tm.fix_params(pm.DiagGauss(output_size=total_size))
    model = tm.variational_bayes(targets, 'inputs', params, priors=prior)
    loss = tm.loss_variational(model)

    # all_params = tm.prox_reparameterize(model['parameters_positive'], tm.softplus, tm.softplus_inv)
    all_params = tm.prox_reparameterize(model['parameters_positive'], track.squareplus, track.squareplus_inv)
    all_params += model['parameters']
    flat = tm.prox_flatten(tm.prox_center(all_params))
    print "%i -> %i" %(n_normflows, len(flat.eval()))

1 -> 5
2 -> 8
3 -> 11
4 -> 14
8 -> 26
20 -> 62
